<a href="https://colab.research.google.com/github/lucaslau77/PO/blob/main/Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install PyPortfolioOpt

In [ ]:
pip install pulp

In [25]:
# Import libraries
from pandas_datareader import data as web
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [26]:
# Get ticker symbols
assets =  ["WYNN", "CRM", "SWKS"]
# assets =  ["CRSR"]

# Assign weights to each stock
weights = np.array([0.1268, 0.1600, 0.3916])
# weights = np.array([1])

In [27]:
# Get the portfolio starting date
stockStartDate = '2013-01-01'

# Get the portfolio ending date
today = datetime.today().strftime('%Y-%m-%d')

In [28]:
# Create a dataframe to store the adjusted close price of stocks
df = pd.DataFrame()

# Store the adjusted close price of each stock into df
for stock in assets:
   df[stock] = web.DataReader(stock, data_source='yahoo', start=stockStartDate, end=today)['Adj Close']

# Show the df
df

,WYNN,CRM,SWKS
Date,,,
2013-01-02,96.279900,42.792500,19.570084
2013-01-03,97.315514,42.177502,19.425457
2013-01-04,98.489761,42.402500,18.937344
2013-01-07,98.824097,42.244999,19.217558
2013-01-08,98.742554,42.492500,18.449221
...,...,...,...
2021-06-11,127.169998,240.309998,171.119995
2021-06-14,124.949997,246.259995,173.830002
2021-06-15,125.400002,242.580002,171.660004


In [29]:
# Calculate the expected returns and the annualised sample covariance matrix of asset returns
mu = expected_returns.mean_historical_return(df) # returns.mean() * 252
S = risk_models.sample_cov(df) # Get the sample covariance matrix

# Optimize for max sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe() # Maximize the Sharpe ratio, and get the raw weights
cleaned_weights = ef.clean_weights() 
print(cleaned_weights) # Note the weights may have some rounding error, meaning they may not add up exactly to 1 but should be close
ef.portfolio_performance(verbose=True)

OrderedDict([('WYNN', 0.0), ('CRM', 0.45746), ('SWKS', 0.54254)])
Expected annual return: 26.4%
Annual volatility: 31.0%
Sharpe Ratio: 0.79


(0.2641887032375266, 0.310344059127536, 0.7868322142979292)

In [30]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
latest_prices = get_latest_prices(df)
weights = cleaned_weights 
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=15000)
allocation, leftover = da.lp_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'CRM': 28, 'SWKS': 47}
Funds remaining: $50.34
